# Course Enrollment

## Authors: Isaac, Conor, and Quoc

    The purpose of this notebook is to teach you, the reader, how to use the code that we have spent the past year creating. The purpose of that code is to predict course enrollment for given quarters. What that means is that you can give our code a large amount of data, which we will teach you how to format and where to put it, and the code will learn the patterns of that code and predict the number of students that will take every class for the quarters that you wish to predict.

In [1]:
# This code block contains all necessary libraries needed to run the code
# Consult the README file to learn how to install the libraries on your machine
import pandas as pd

In [2]:
# This is the data that we use to train our model
# Make sure your data is in this format, we got this from the Registrar's office
base_data = pd.read_csv('data/WWU_course_info.csv', encoding='ISO-8859-1')
base_data.head()

,TERM,CRN,SUBJECT,COURSE_NUMBER,TITLE,ACTUAL_ENROLL,CAPENROLL,PRIMARY_BEGIN_TIME,PRIMARY_END_TIME,U,M,T,W,R,F,S,PRIMARY_INSTRUCTOR_TENURE_CODE,CAMPUS
0,201510,10046,COMM,442,Video Workshop,12,16,1700.0,1820.0,NaN,NaN,T,NaN,R,NaN,NaN,NT,M
1,201510,10049,ENG,101,Writing and Critical Inquiry,24,24,830.0,950.0,NaN,M,NaN,NaN,NaN,F,NaN,NaN,M
2,201510,10052,ENG,101,Writing and Critical Inquiry,24,24,830.0,950.0,NaN,M,NaN,W,NaN,NaN,NaN,NaN,M
3,201510,10055,ECON,206,Intro to Microeconomics,60,60,1130.0,1250.0,NaN,M,NaN,W,NaN,F,NaN,NT,M
4,201510,10060,ENG,101,Writing and Critical Inquiry,24,24,1000.0,1120.0,NaN,M,NaN,NaN,NaN,F,NaN,NaN,M


In [3]:
from curation import Curate

In [4]:
Curate.main()

In [6]:
pd.read_csv("data/machine_learning_data.csv")

,COURSE_NUMBER,ACTUAL_ENROLL,U,M,T,W,R,F,S,TERM_Fall,...,CAPENROLL_65,CAPENROLL_70,CAPENROLL_75,CAPENROLL_80,CAPENROLL_85,CAPENROLL_90,CAPENROLL_100,BEGIN_TIME_MINUTES,sin_time,cos_time
0,101.0,13,0,0,0,0,0,0,0,False,...,False,False,False,False,False,False,False,9.0,0.923880,3.826834e-01
1,102.0,18,0,0,0,0,0,0,0,False,...,False,False,False,False,False,False,False,13.0,0.991445,-1.305262e-01
2,140.0,25,0,0,0,0,0,0,0,True,...,False,False,False,False,False,False,False,8.0,0.866025,5.000000e-01
3,141.0,25,0,0,0,0,0,0,0,True,...,False,False,False,False,False,False,False,12.0,1.000000,6.123234e-17
4,301.0,20,0,0,0,0,0,0,0,True,...,False,False,False,False,False,False,False,12.0,1.000000,6.123234e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,247.0,18,0,0,0,0,0,0,0,False,...,False,False,False,False,False,False,False,10.0,0.965926,2.588190e-01
1849,301.0,20,0,0,0,0,0,0,0,False,...,False,False,False,False,False,False,False,11.0,0.991445,1.305262e-01
1850,591.0,21,0,0,1,0,0,0,0,False,...,False,False,False,False,False,False,False,16.0,0.866025,-5.000000e-01
1851,347.0,21,0,0,0,0,0,0,0,False,...,False,False,False,False,False,False,False,9.0,0.923880,3.826834e-01
